# Using Large Language Models

## Working with Text Models

In [62]:
# Imports

import os
from langchain_openai import OpenAI

Note, that LangChain automatically looks up for any environment variable with the name OPENAI_API_KEY automatically when making a connection to OpenAI.

In [63]:
# Initialize the Text Model

api_key = os.getenv("OPENAI_API_KEY")
llm = OpenAI(openai_api_key = api_key)

In [64]:
# Lets work on a simplest way to get a text autocomplete -
print(llm.invoke("Here is a fun fact about Pluto Planet :"))



Pluto was discovered in 1930 by American astronomer Clyde Tombaugh, who was only 24 years old at the time. He spent nearly a year comparing photographs of the night sky, looking for a ninth planet predicted by astronomer Percival Lowell. Tombaugh ultimately found Pluto by noticing a moving object in the photographs that did not match the expected background stars.


In [65]:
# Lets generate a full output with mmore information
result = llm.generate(["Here is Fun Fact about Pluto : ",
                       "Here is a Fun Fact about Mars :"])

# to check the schema of the response
result.model_json_schema()

{'$defs': {'BaseMessage': {'additionalProperties': True,
   'description': 'Base abstract message class.\n\nMessages are the inputs and outputs of ChatModels.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'title': 'Type', 'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Id'}},
   'required': ['content', 'type'],
   'title': 'BaseMessage',
   'type': 'object'},
  'BaseMessageChunk': {'additionalProperties': True,
   'description': 'Message chunk, which can be concatenated with other Message chunks.',
   'properties': {'co

In [67]:
# Check the detailed response
response = result.generations
response[0][0].text

"\n\nPluto's largest moon, Charon, is almost half the size of Pluto itself. This makes it the largest moon relative to its planet in our solar system."

In [49]:
# Check the token usage and other details about output of the LLM

result.llm_output

{'token_usage': {'total_tokens': 112,
  'prompt_tokens': 16,
  'completion_tokens': 96},
 'model_name': 'gpt-3.5-turbo-instruct'}

## Working with Chat Models

In [52]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(openai_api_key = api_key)

In [56]:
# now lets ask a question
result = chat.invoke([HumanMessage(content="Can you tell me a fact about Earth ?")])
result

AIMessage(content='Sure! One fact about Earth is that it is the only planet in our solar system known to support life.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 16, 'total_tokens': 38, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3d257724-fdbc-4f09-9885-99dd9a14af86-0', usage_metadata={'input_tokens': 16, 'output_tokens': 22, 'total_tokens': 38})

In [57]:
# Get the text response
result.content

'Sure! One fact about Earth is that it is the only planet in our solar system known to support life.'

In [68]:
# Lets try another
result = chat.generate([[SystemMessage(content="You are a University Professor"),
                         HumanMessage(content="Can you tell me a fact about Earth ?")]])

result.generations[0][0].text

'Certainly! One interesting fact about Earth is that it is the only known planet in our solar system that has liquid water on its surface, which is essential for supporting life as we know it.'

- Temperature is a parameter that controls the “creativity” or randomness of the text generated by open ai llm. A higher temperature (e.g., 0.7) results in more diverse and creative output, while a lower temperature (e.g., 0.2) makes the output more deterministic and focused.

- In practice, temperature affects the probability distribution over the possible tokens at each step of the generation process. A temperature of 0 would make the model completely deterministic, always choosing the most likely token.

In [71]:
# We can add some extra parameters and args - here we are adding some extreme values
result = chat.invoke([HumanMessage(content="Can you tell me a fact about Earth ?")],
                     temperature=2, max_tokens=100)
result.content

"Sure, here is a fact about Earth :\n\nApproximately 71% of Earth's surface is covered in water, making it unique among the planets in our solar system."

## Caching

Should only do this if the prompt is the exact same and the historical replies are okay to return.

In [72]:
import langchain
from langchain.cache import InMemoryCache

# Initialize the cache
langchain.llm_cache = InMemoryCache()

In [73]:
# The first time it is not in cache and hence might take little time
llm.invoke("Tell me fact about Mars")

'\n\n1. Mars is the fourth planet from the sun in our solar system, located between Earth and Jupiter.\n\n2. The planet is named after the Roman god of war, likely due to its reddish appearance, which is caused by iron oxide (rust) on its surface.\n\n3. Mars has a thin atmosphere composed mainly of carbon dioxide, with traces of nitrogen and argon.\n\n4. The planet has two small moons, Phobos and Deimos, which are thought to be captured asteroids.\n\n5. Mars is often called the "Red Planet" because of its rusty color, but it also has white polar ice caps made of frozen carbon dioxide and water.\n\n6. The largest volcano in the solar system, Olympus Mons, is located on Mars and stands at a height of 22 km (14 miles).\n\n7. Mars has a day and night cycle similar to Earth, with a day lasting approximately 24 hours and 37 minutes.\n\n8. The average temperature on Mars is around -80°F (-62°C), but it can reach highs of 68°F (20°C) at the equator during summer.\n\n9. Evidence suggests that M

In [74]:
# This response should come very fast
llm.invoke("Tell me fact about Mars")

'\n\n1. Mars is the fourth planet from the sun in our solar system, located between Earth and Jupiter.\n\n2. The planet is named after the Roman god of war, likely due to its reddish appearance, which is caused by iron oxide (rust) on its surface.\n\n3. Mars has a thin atmosphere composed mainly of carbon dioxide, with traces of nitrogen and argon.\n\n4. The planet has two small moons, Phobos and Deimos, which are thought to be captured asteroids.\n\n5. Mars is often called the "Red Planet" because of its rusty color, but it also has white polar ice caps made of frozen carbon dioxide and water.\n\n6. The largest volcano in the solar system, Olympus Mons, is located on Mars and stands at a height of 22 km (14 miles).\n\n7. Mars has a day and night cycle similar to Earth, with a day lasting approximately 24 hours and 37 minutes.\n\n8. The average temperature on Mars is around -80°F (-62°C), but it can reach highs of 68°F (20°C) at the equator during summer.\n\n9. Evidence suggests that M